Procesamiento del Lenguaje Natural
===
Lab 3.1 - Implementación personalizada de CountVectorizer al Español
===
Alumno: Gonzalo Bertinat

Legajo: 160.615-3

Curso: K3572

-------------------------------

Enunciado:

_Subir a tu github una implementación personalizada de NLTK para CountVectorizer
que haga steam y stopwords del idioma español y dos ejemplos de oraciones usando tu clase.
También importá un corpus como 20_Newsdataset pero que esté en español. Qué
corpus poner, queda a tu criterio!_

Bien, para resolver el enunciado necesitamos nuestra propia implementación del CountVectorizer, por lo que vamos a crear una clase que herede de CountVectorizer como en el Lab 3.

Antes que nada estaría bueno saber algunas de las Stop Words que existen en español.
Si hacemos como se nos enseño en el lab deberíamos ejecutar el siguiente código:


In [1]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = 'spanish')

vectorizer.get_stop_words()

ValueError: not a built-in stop list: spanish

Ups, nos encontramos con un error, si prestamos atención abajo nos dice que:

_"ValueError: not a built-in stop list: spanish"_

Esto sucede porque 'spanish' no está contemplado como valor posible al instanciar un CountVectorizer. 
Si revisamos la documentación de SciKit learn: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Vemos que nos dice en los parámetros del constructor el detalle del parámetro **stop_words**:

_"stop_words : string {‘english’}, list, or None (default)"_

La documentación oficial nos comenta que stop_words puede valer el string literal "english" (que es como se venía haciendo hasta ahora), una lista o None (sería sin enviarle el valor en el constructor).

Como "spanish" no es el string "english" ni es una lista, no podemos hacerlo como en el código escrito recién.

La forma correcta será usando la opción del parámetro en forma de lista, y en esa lista enviaremos las palabras Stop Words del idioma español.

Pero.. ¿de dónde las sacamos? De nuestro queridísimo NLTK.
Con el código siguiente obtenemos las Stop Words del español y mostramos las primeras 20:

In [2]:
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')

print(spanish_stopwords[:20])

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo']


Ahora sí vamos a poder crear nuestra clase e instanciarla con idioma español:

In [3]:
# Importamos Stem de NLTK para hacer stemming
import nltk.stem

# Importamos el CountVectorizer original para hacer una sobreescritura
from sklearn.feature_extraction.text import CountVectorizer

# Creamos un stemmer de NLTK para idioma Español, en este caso sí está contemplado el string "spanish" !
spanish_stemmer = nltk.stem.SnowballStemmer('spanish')

# Definición de la clase, que hereda de CountVectorizer
class StemmedSpanishCountVectorizer(CountVectorizer):
    
    # Redefinición del build_analyzer
    def build_analyzer(self):
        analyzer = super(StemmedSpanishCountVectorizer,self).build_analyzer()
        # Aplicamos el Steammer de NLTK a la salida del build_analyzer
        return lambda doc: (spanish_stemmer.stem(w) for w in analyzer(doc))

Instaciamos:

In [4]:
# En este caso enviamos de parámetro de stop_words la lista que obtuvimos recién
stem_vectorizer = StemmedSpanishCountVectorizer(min_df=1, stop_words=spanish_stopwords)
stem_analyze = stem_vectorizer.build_analyzer()

Bueno, el enunciado nos pide que usemos dos oraciones de ejemplo, y luego un corpus en español.

Vamos con lo primero: declaremos dos oraciones.

In [5]:
frase1 = "A Juan le gusta mucho salir a correr."
frase2 = "La comida preferida de María es la pizza."

Ahora vamos a obtener el steam (raíz) de las palabras de cada frase ignorando las Stop Words:

In [6]:
X = stem_analyze(frase1)
for stem in X:
    print(stem)

juan
gust
sal
corr


In [7]:
Y = stem_analyze(frase2)
for stem in Y:
    print(stem)

com
prefer
mar
pizz


Ahora importemos un corpus en Español:

In [8]:
from nltk.corpus import cess_esp

Extraigamos las primeras 70 palabras, para poder visualizar y obtener dos oraciones de este corpus:

In [9]:
cess_esp.words()[:70]

['El',
 'grupo',
 'estatal',
 'Electricité_de_France',
 '-Fpa-',
 'EDF',
 '-Fpt-',
 'anunció',
 'hoy',
 ',',
 'jueves',
 ',',
 'la',
 'compra',
 'del',
 '51_por_ciento',
 'de',
 'la',
 'empresa',
 'mexicana',
 'Electricidad_Águila_de_Altamira',
 '-Fpa-',
 'EAA',
 '-Fpt-',
 ',',
 'creada',
 'por',
 'el',
 'japonés',
 'Mitsubishi_Corporation',
 'para',
 'poner_en_marcha',
 'una',
 'central',
 'de',
 'gas',
 'de',
 '495',
 'megavatios',
 '.',
 'Una',
 'portavoz',
 'de',
 'EDF',
 'explicó',
 'a',
 'EFE',
 'que',
 'el',
 'proyecto',
 'para',
 'la',
 'construcción',
 'de',
 'Altamira_2',
 ',',
 'al',
 'norte',
 'de',
 'Tampico',
 ',',
 'prevé',
 'la',
 'utilización',
 'de',
 'gas',
 'natural',
 'como',
 'combustible',
 'principal']

In [10]:
# "anunció hoy, jueves, la compra del 51_por_ciento de la empresa mexicana"
cess_frase1_words = cess_esp.words()[7:21]
# "Una portavoz de EDF explicó a EFE que el proyecto para la construcción de Altamira_2, al norte de Tampico, prevé la utilización de gas natural como combustible principal"
cess_frase2_words = cess_esp.words()[40:70]

# Convertimos las listas de palabras a una cadena:
cess_frase1 = " ".join(str(x) for x in cess_frase1_words)
cess_frase2 = " ".join(str(x) for x in cess_frase2_words)

In [11]:
# Hacemos el stemming
X = stem_analyze(cess_frase1)
for stem in X:
    print(stem)

anunc
hoy
juev
compr
51_por_cient
empres
mexican
electricidad_aguila_de_altamir


In [12]:
# Lo mismo con la segunda frase
Y = stem_analyze(cess_frase1)
for stem in Y:
    print(stem)

anunc
hoy
juev
compr
51_por_cient
empres
mexican
electricidad_aguila_de_altamir


Podemos concluir que el steam en español de SnowballStemmer hace el steamming aplicando el método del truncado, quita letras del final, asumiendo que las primeras letras son la raíz (steam) de la palabra.